# Morphological operations and Non-Maximum Suppression (NMS) for document element detection

## 1. Import the necessary packages.

In [1]:
# For computer vision
import cv2

# For logging
from logging import FileHandler
from vlogging import VisualRecord
import logging

# For connected-component analysis
from skimage.filters import threshold_adaptive
from skimage import measure

# For connected-component analysis and Non-Maximum Suppression
import numpy as np

Open the logging file and set the attributes.

In [2]:
logger = logging.getLogger("detect_elements")
fh = FileHandler("detect_elements_log.html", mode = "w")

logger.setLevel(logging.DEBUG)
logger.addHandler(fh)

# Prevent logger output in IPython
logger.propagate = False

# Define a function to handle visual logging
def vlog(image, title):
    logger.debug(VisualRecord(title, image, fmt = "png"))

## 2. Define functions.

Implement fast Non-Maximum Suppression from PyImageSearch.

http://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

In [3]:
def non_maximum_suppression(boxes, overlapThresh):
    # Return an empty list if there are no boxes.
    if len(boxes) == 0:
        return []
    
    # If the bounding boxes are integers, convert them to floats: floats are needed for divisions.
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
        
    # Initialize a list of picked indexes.
    pick = []
    
    # Grab the coordinates of the bounding boxes.
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    
    # Compute the area of the bounding boxes and sort them by the bottom-right y-coordinate.
    area = (x2 - x1 +1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    # Keep looping while some indexes remain in the list of indexes
    while len(idxs) > 0:
        # Grab the last index in the list and add the index value to the list of picked indexes
        last = len(idxs) -1
        i = idxs[last]
        pick.append(i)
        
        # Find the largest (x, y) coordinates for the start of the bounding box and the smallest (x, y) coordinates for the end of the bounding box.
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]]) 
        
        # Compute the width and height of the bounding box.
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)  
        
        # Compute the ratio of overlap.
        overlap = (w * h) / area[idxs[:last]]
        
        # Delete all indexes from the index list that overlap the overlap threshold.
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
        
    # Return the picked bounding boxes as integers.
    return boxes[pick].astype("int")

## 3. Prepare the document image.

TO DO BEFORE TESTING:

1. Set up an argument parser
2. Grab the images from a directory
3. Write the results on disk
4. Loop over the results and prompt for the number of false positives

Load the image.

In [4]:
image = cv2.imread('test_images/1967_hft_side_b_lowres_w1200.jpg')

# Logging
logger.debug("Image width: {}, height: {}".format(image.shape[1], image.shape[0]))
vlog(image, "Original image")

Convert image to grayscale.

In [5]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

vlog(gray, "Grayscale")

Apply bilateral filtering to remove detail but preserve edges.

In [6]:
params = (11, 41, 21)
blurred = cv2.bilateralFilter(gray, params[0], params[1], params[2])

# Logging
logger.debug("Parameters for bilateral filtering: diameter of the pixel neighbourhood: {}, standard deviation for color: {}, standard deviation for space: {}".format(params[0], params[1], params[2]))
vlog(blurred, "Bilaterally filtered")

Define a kernel size for morphological operations.

> The kernel size must be determined after deciding input image resolution. It should be based on type size and correspond roughly to the x-height of the font face used for body text.

In [7]:
kernelsize = (5, 11)

Perform Otsu's thresholding.

In [8]:
(T, thresholded) = cv2.threshold(blurred, 0, 255, cv2.THRESH_OTSU)

# Logging
logger.debug("Otsu's threshold: {}".format(T))
vlog(thresholded, "Thresholded")

## 4. Perform morphological operations.

In [9]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelsize)

#### Morphological gradient 

In [10]:
gradient = cv2.morphologyEx(thresholded.copy(), cv2.MORPH_GRADIENT, kernel)

# Logging
logger.debug("Kernel size: {}".format(kernelsize))
vlog(gradient, "Morphological gradient applied")

#### Erode

In [11]:
eroded = cv2.erode(gradient, None, iterations = 1)

# Logging
vlog(eroded, "Morphological gradient eroded")

## 5. Perform connected-components labeling

Perform connected component labeling and set up a mask for the labels to be kept.

In [12]:
labels = measure.label(eroded, neighbors = 8, background = 0)

gradient_mask = np.zeros(gradient.shape, dtype = "uint8")

Loop over the labels twice:
    1. Calculate the average number of pixels per label.
    2. Decide which labels to include in the mask.

In [13]:
# First loop

numpixels_all = []

for (i, label) in enumerate(np.unique(labels)):
    if label == -1:
        continue
    labelmask = np.zeros(gradient.shape, dtype = "uint8")
    labelmask[labels == label] = 255
    numpixels = cv2.countNonZero(labelmask)
    numpixels_all.append(numpixels)

average = sum(numpixels_all) / len(numpixels_all)

In [14]:
# Second loop

for (i, label) in enumerate(np.unique(labels)):
    if label == -1:
        continue
    labelmask = np.zeros(gradient.shape, dtype = "uint8")
    labelmask[labels == label] = 255
    numpixels = cv2.countNonZero(labelmask)
    
    if numpixels > (int(average) * 0.05):
        gradient_mask = cv2.add(gradient_mask, labelmask)
        
# Logging
logger.debug("Average size for label: {}".format(average))   
vlog(gradient_mask, "Mask for morphological gradient after connected-components labeling")

## 6. Find contours in the processed image

Find contours in the image after applying morphological gradient and performing connected-components labeling.

In [15]:
(contours, hierarchy) = cv2.findContours(gradient_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

Set up another mask.

In [16]:
contour_mask = np.zeros(gradient_mask.shape, dtype = "uint8")

Draw contours on the mask.

In [17]:
for c in contours:
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(contour_mask, (x, y), (x + w, y + h), (255, 255, 255), -1)

# Logging
vlog(contour_mask, "Contour mask")

Detect contours in the mask and draw them on the original image.

In [18]:
(maskcontours, maskhierarchy) = cv2.findContours(contour_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [19]:
original = image.copy()

for mc in maskcontours:
    (x, y, w, h) = cv2.boundingRect(mc)
    cv2.rectangle(original, (x, y), (x + w, y + h), (0, 0, 255), 1)
    
vlog(original, "RESULT 1: Contours detected in the contour mask")

In [20]:
clone = image.copy()

for c, h in zip(contours, hierarchy[0]):
    area = cv2.contourArea(c)
    if h[:][3] == -1 and area >= 25: # Is the hierarchy still needed?
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(clone, (x, y), (x + w, y + h), (0, 0, 255), 1)
        
vlog(clone, "RESULT 2: Contours detected in the original image (morphological gradient + connected-components labeling)")

## 7. Apply Non-Maximum Suppression

Loop over the contours to get the bounding boxes.

In [21]:
bounding_boxes = []

for c in contours:
    (x, y, w, h) = cv2.boundingRect(c)
    bbox = (x, y, x + w, y + h) # NMS requires a bounding box, not just coordinates!
    bounding_boxes.append(bbox)

Apply Non-Maximum Suppression.

In [22]:
bounding_boxes_nms = non_maximum_suppression(np.asarray(bounding_boxes), 0.5)

Draw the remaining contours.

In [23]:
nms = image.copy()

for (startx, starty, endx, endy) in bounding_boxes_nms:
    cv2.rectangle(nms, (startx, starty), (endx, endy), (0, 0, 255), 1)
    
vlog(nms, "RESULT 3: Non-Maximum Suppression applied to RESULT 2")